In [1]:
%%capture
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [2]:
iOption0 = 0
Save_to_Html = False
Save_to_pdf = False

In [3]:
N_centr = 10
N_real_centr = 5
eIDbins = [1,2,3,4,5]
primaryID = [[i for i in range(1,26)],[19,20,24,25],[20,25]]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
real_file_path = 'input/' 
real_file_names=['my-m_ee_Run14AuAu_55th_new_19657_1072runs.root']
#m_ee_Run14AuAu_49th_new_19636_1072runs m_ee_Run14AuAu_50th_new_19640_1072runs my-m_ee_Run14AuAu_54th_new_19654_1072runs m_ee_Run14AuAu_55th_new_19657_1072runs
#m_ee_Run14AuAu_45th_new_19617_240runs , m_ee_Run14AuAu_44th_new_19614_975runs, m_ee_Run14AuAu_46th_new_19619_999runs m_ee_Run14AuAu_47th_new_19623_1078runs
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v6.root']
hadron_file_names=['charged_pi_100M_v3.root']
dalitz_file_names=['dalitz_50M_v7.root']
conv_file_names=['conv_100M.root']
sim_file_names = [primary_file_names,hadron_file_names,dalitz_file_names,conv_file_names ]
sim_file_names = [dalitz_file_names, hadron_file_names, primary_file_names ]
part_names = ["c#bar{c}","dalitz","#gamma #rightarrow ee","#pi^{-}"]
eID_names = ["loose","tight","BDT loose","BDT basic","BDT tight"]
hist_names = ["el_pt_hist_"]
hist_pt_orig_name = "hist_pt_orig"

In [4]:
hists_read_real, hists_read_sim = [], []

for iFilereal in range(len(real_file_names)):
    infile = root.TFile.Open(real_file_path+real_file_names[iFilereal], "read")
    hists_read_real_ifile = []
    for icentr in range(N_centr):
        hists_read_real_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_real_ifile[-1].SetDirectory(root.nullptr)
        hists_read_real_ifile[-1].SetName(hist_names[0]+f'{icentr}'+'_real')
    hists_read_real.append(hists_read_real_ifile)
    infile.Close()

for iFilesim in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFilesim][0], "read")
    hists_read_sim_ifile = []
    for icentr in range(N_centr):
        hists_read_sim_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_sim_ifile[-1].SetDirectory(root.nullptr)
        hists_read_sim_ifile[-1].SetName(hist_names[0]+f'{icentr}'+f'_sim{iFilesim}_')
    hists_read_sim.append(hists_read_sim_ifile)
    if iFilesim == 0:   
        hist_pt_orig = infile.Get(hist_pt_orig_name)
        hist_pt_orig.SetDirectory(root.nullptr)
    infile.Close()

In [5]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [6]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
iED = 4
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.0, 4.4))
    h[-1].SetMaximum(1.2* max(hists_pt_real[0][iED][0][icentr].GetMaximum(),hists_pt_real[0][iED][0][icentr+5].GetMaximum()))
    h[-1].SetMinimum(10)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    
    Format_Hist_total(hists_pt_real[0][iED][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_real[0][iED][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_real[0][iED][0][icentr].Draw("same")
    hists_pt_real[0][iED][0][icentr+5].Draw("same")
    legends.append(root.TLegend(0.5,0.7,0.9,0.9))
    legends[-1].AddEntry(hists_pt_real[0][iED][0][icentr],"pos","l")
    legends[-1].AddEntry(hists_pt_real[0][iED][0][icentr+5],"neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
if not Save_to_Html: c0.Draw()

In [7]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
ipart = 0
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(1.2* max(hists_pt_sim[ipart][0][0][icentr].GetMaximum(),hists_pt_sim[ipart][0][0][icentr+5].GetMaximum()))
    h[-1].SetMinimum(1000)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_sim[ipart][0][0][icentr].Draw("same")
    hists_pt_sim[ipart][0][0][icentr+5].Draw("same")
    hists_pt_sim[ipart][1][0][icentr].Draw("same")
    hists_pt_sim[ipart][2][0][icentr].Draw("same")
    hists_pt_sim[ipart][3][0][icentr].Draw("same")
    hists_pt_sim[ipart][4][0][icentr].Draw("same")
    legends.append(root.TLegend(0.5,0.7,0.9,0.9))
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr],"pos","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr+5],"neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
if not Save_to_Html: c0.Draw()

In [8]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
ipart = 0
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(1.9)
    h[-1].SetMinimum(0.3)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="Cut/Loose", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=0,  Lalpha=0)
    
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    #root.gPad.SetLogy()
    h[-1].Draw()
    
    hists_pt_sim[ipart][1][0][icentr].Divide(hists_pt_sim[ipart][0][0][icentr])
    hists_pt_sim[ipart][2][0][icentr].Divide(hists_pt_sim[ipart][0][0][icentr])
    hists_pt_sim[ipart][3][0][icentr].Divide(hists_pt_sim[ipart][0][0][icentr])
    hists_pt_sim[ipart][4][0][icentr].Divide(hists_pt_sim[ipart][0][0][icentr])
    hists_pt_sim[ipart][1][0][icentr].Draw("same")
    hists_pt_sim[ipart][2][0][icentr].Draw("same")
    hists_pt_sim[ipart][3][0][icentr].Draw("same")
    hists_pt_sim[ipart][4][0][icentr].Draw("same")
    Format_Graph(hists_pt_sim[ipart][1][0][icentr],Lcolor = 1, Lwidth=3)
    Format_Graph(hists_pt_sim[ipart][2][0][icentr],Lcolor = 2, Lwidth=3)
    Format_Graph(hists_pt_sim[ipart][3][0][icentr],Lcolor = 4, Lwidth=3)
    Format_Graph(hists_pt_sim[ipart][4][0][icentr],Lcolor = 6, Lwidth=3)
    legends.append(root.TLegend(0.5,0.7,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]} %"))
    legends[-1].AddEntry(hists_pt_sim[ipart][1][0][icentr],"std cut","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][2][0][icentr],"BDT loose","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][3][0][icentr],"BDT basic","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][4][0][icentr],"BDT tight","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
if not Save_to_Html: c0.Draw()
if Save_to_pdf: c0.SaveAs("output/note/ele_eff.pdf")

In [9]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
ipart = 1
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}{ipart}", f"name{icentr}{ieID}{iconv}{ipart}", 100, 0.4, 4.9))
    h[-1].SetMaximum(10000 - 1)
    h[-1].SetMinimum(0.01)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="Cut/BDT", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=0,  Lalpha=0)
    
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_sim[ipart][0][0][icentr].Rebin(4)
    hists_pt_sim[ipart][1][0][icentr].Rebin(4)
    hists_pt_sim[ipart][2][0][icentr].Rebin(4)
    hists_pt_sim[ipart][3][0][icentr].Rebin(4)
    hists_pt_sim[ipart][4][0][icentr].Rebin(4)
    
    hists_pt_sim[ipart][0][0][icentr].Divide(hists_pt_sim[ipart][4][0][icentr])
    hists_pt_sim[ipart][1][0][icentr].Divide(hists_pt_sim[ipart][4][0][icentr])
    hists_pt_sim[ipart][2][0][icentr].Divide(hists_pt_sim[ipart][4][0][icentr])
    hists_pt_sim[ipart][3][0][icentr].Divide(hists_pt_sim[ipart][4][0][icentr])
    hists_pt_sim[ipart][0][0][icentr].Draw("same")
    hists_pt_sim[ipart][1][0][icentr].Draw("same")
    hists_pt_sim[ipart][2][0][icentr].Draw("same")
    hists_pt_sim[ipart][3][0][icentr].Draw("same")
    Format_Graph(hists_pt_sim[ipart][0][0][icentr],Lcolor = 6, Lwidth=3)
    Format_Graph(hists_pt_sim[ipart][1][0][icentr],Lcolor = 1, Lwidth=3)
    Format_Graph(hists_pt_sim[ipart][2][0][icentr],Lcolor = 2, Lwidth=3)
    Format_Graph(hists_pt_sim[ipart][3][0][icentr],Lcolor = 4, Lwidth=3)
    legends.append(root.TLegend(0.5,0.7,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]} %"))
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr],"loose cut","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][1][0][icentr],"std cut","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][2][0][icentr],"BDT loose","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][3][0][icentr],"BDT basic","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
if not Save_to_Html: c0.Draw()
if Save_to_pdf: c0.SaveAs("output/note/had_rej.pdf")

In [10]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
ipart = 1
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(1.2* max(hists_pt_sim[ipart][0][0][icentr].GetMaximum(),hists_pt_sim[ipart][0][0][icentr+5].GetMaximum()))
    h[-1].SetMinimum(1)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    Format_Hist_total(hists_pt_sim[ipart][0][0][icentr+5], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    root.gPad.SetLogy()
    h[-1].Draw()
    hists_pt_sim[ipart][0][0][icentr].Draw("same")
    hists_pt_sim[ipart][0][0][icentr+5].Draw("same")
    legends.append(root.TLegend(0.5,0.7,0.9,0.9))
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr],"pos","l")
    legends[-1].AddEntry(hists_pt_sim[ipart][0][0][icentr+5],"neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
if not Save_to_Html: c0.Draw()

In [11]:
idivide = 1

In [12]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(0,len(eIDbins)):
        Format_Hist_total(hists_pt_real[0][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_real[0][eID][0][icentr].Divide(hists_pt_real[0][eID][0][icentr+5])
        hists_pt_real[0][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_real[0][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

if not Save_to_Html: c0.Draw()

In [13]:
iFile = 0
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(0,len(eIDbins)):
        Format_Hist_total(hists_pt_sim[iFile][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_sim[iFile][eID][0][icentr].Divide(hists_pt_sim[iFile][eID][0][icentr+5])
        hists_pt_sim[iFile][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_sim[iFile][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

if not Save_to_Html: c0.Draw()

In [14]:
iFile = 1
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(1,len(eIDbins)):
        Format_Hist_total(hists_pt_sim[iFile][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_sim[iFile][eID][0][icentr].Divide(hists_pt_sim[iFile][eID][0][icentr+5])
        hists_pt_sim[iFile][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_sim[iFile][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

if not Save_to_Html: c0.Draw()

In [15]:
c0 = root.TCanvas(f"c0",f"c0",1800,1200)
c0.Divide(3,2)
h = []
legends = []
for icentr in range(N_real_centr):
    c0.cd(icentr+1)
    h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
    h[-1].SetMaximum(2)
    h[-1].SetMinimum(0.5)
    Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                          Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
    h[-1].Draw()
    legends.append(root.TLegend(0.5,0.7,0.9,0.9)) 
    Format_Legend(legends[-1])
    for eID in range(0,len(eIDbins)):
        Format_Hist_total(hists_pt_real[0][eID][0][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
              Mstyle=21,  Msize=0, Mcolor=colors[eID],  Lwidth=6,  Lcolor=colors[eID],  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        if idivide : hists_pt_real[0][eID][0][icentr].Divide(hists_pt_sim[0][eID][0][icentr])
        hists_pt_real[0][eID][0][icentr].Draw("same")
        legends[-1].AddEntry(hists_pt_real[0][eID][0][icentr],f"{eID_names[eID]}","l")
    legends[-1].Draw()

if not Save_to_Html:  c0.Draw()
idivide = 0

In [16]:
if False:
    c0 = root.TCanvas(f"c0",f"c0",1400,1400)
    c0.Divide(2,3)
    for icentr in range(N_centr):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        hists_pt_sim[1][0][0][icentr].Draw()
    c0.Draw()

In [17]:
rebin = 1
for icentr in range(N_centr):
    for ieID in range(0,len(eIDbins)):
        for iconv in range(len(primaryID)):
            hists_pt_real[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[1][ieID][iconv][icentr].Rebin(rebin)

In [18]:
graphs= [] 
refrerence  = 3
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)):
        if eID == refrerence:continue
        graphs_eid = []
        for iconv in range(len(primaryID)):
            graphs_conv = []
            for iptbin in range(1,1+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                raw_signal_frac =  hists_pt_real[0][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_real[0][refrerence][iconv][icentr].GetBinContent(iptbin)+1e-6)
                raw_primary_frac = hists_pt_sim[0][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][refrerence][iconv][icentr].GetBinContent(iptbin)+1e-6)
                raw_hadron_frac =  hists_pt_sim[1][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[1][refrerence][iconv][icentr].GetBinContent(iptbin)+1e-6)
                #print(raw_signal_frac, raw_primary_frac, raw_hadron_frac)
                if (raw_primary_frac - raw_signal_frac) == 0: hadron_frac = 0
                else: 
                    if raw_primary_frac - raw_hadron_frac > 0:
                        hadron_frac = ( raw_primary_frac - raw_signal_frac ) / ( raw_primary_frac - raw_hadron_frac )
                    else: hadron_frac = -1
                    if hadron_frac<0: hadron_frac = 0
                    if hadron_frac>1: hadron_frac = 1
                hadron_frac_err = hadron_frac *0.1+0.05 #(hists_pt_sim[1][ieID][iconv][icentr].GetBinError(iptbin)/(hists_pt_sim[1][ieID][iconv][icentr].GetBinContent(iptbin)+1e-6))*2**0.5
                    #print(hadron_frac)
                graphs_conv.append([hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin),hadron_frac,hadron_frac_err])
            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
if not Save_to_Html: print(graphs)

[[[[[0.05, 0, 0.05], [0.15000000000000002, 1, 0.15000000000000002], [0.25, 1, 0.15000000000000002], [0.35000000000000003, 1, 0.15000000000000002], [0.45, 1, 0.15000000000000002], [0.55, 1, 0.15000000000000002], [0.6500000000000001, 0.9999999983965927, 0.14999999983965928], [0.7500000000000001, 1, 0.15000000000000002], [0.8500000000000001, 0.9999999985095475, 0.14999999985095475], [0.9500000000000001, 0.9999999984391186, 0.14999999984391188], [1.05, 0.9999999983542094, 0.14999999983542095], [1.1500000000000001, 0.999999998208854, 0.1499999998208854], [1.2500000000000002, 0.9999999980689581, 0.14999999980689582], [1.35, 0.9999999979214389, 0.1499999997921439], [1.4500000000000002, 0.9999999977164681, 0.1499999997716468], [1.55, 0.9999999972931927, 0.14999999972931927], [1.6500000000000001, 0.9999999969196893, 0.14999999969196892], [1.7500000000000002, 0.9999999969104533, 0.14999999969104533], [1.85, 0.9999999964543627, 0.14999999964543625], [1.9500000000000002, 0.9999999957325133, 0.1499

In [19]:
Tgraphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)-1):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            graphs_conv = root.TGraphErrors()
            ipoint = 0
            for iptbin in range(0,0+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                if hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1)<0.4: continue
                if hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1)>4.4: continue
                if graphs[icentr][ieID][iconv][iptbin][1]<0.01: continue
                graphs_conv.AddPoint(hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1),graphs[icentr][ieID][iconv][iptbin][1])
                graphs_conv.SetPointError(ipoint,hists_pt_real[0][ieID][iconv][icentr].GetBinWidth(iptbin+1)/2,graphs[icentr][ieID][iconv][iptbin][2])
                ipoint+=1

            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    Tgraphs.append(graphs_centr)

In [20]:
only_pos = 1
c0 = root.TCanvas(f"c0",f"c0",1400,1400*(2-only_pos))
N_centr_loc = N_centr - 5*only_pos
c0.Divide(3,3*(2-only_pos))
h = []
TMultigraphs = []
for icentr in range(N_centr_loc):
    graphs_centr = []
    for iconv in range(1,len(primaryID)):
        c0.cd(icentr+1+N_centr_loc*(iconv-1))
        TMultigraphs.append(root.TMultiGraph())
        for ieID in range(0,len(eIDbins)-1):
            graphs_eid = []
            if ieID == 0 :
                h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.4, 4.9))
                h[-1].SetMaximum(0.99)
                Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
                h[-1].Draw()
                Format_Graph( Tgraphs[icentr][ieID][iconv],  Mstyle=21,  Msize=1,  Mcolor=1,  Lwidth=1,  Lcolor=1,  Malpha=1,  Lalpha=1)
                Tgraphs[icentr][ieID][iconv].Draw("same P")
            else :
                Format_Graph( Tgraphs[icentr][ieID][iconv],  Mstyle=20,  Msize=1,  Mcolor=2+ieID,  Lwidth=1,  Lcolor=1,  Malpha=2,  Lalpha=1)
                Tgraphs[icentr][ieID][iconv].Draw("same P")
            #TMultigraphs[-1].Add(Tgraphs[icentr][ieID][iconv])
            if ieID == len(eIDbins)-4: Tgraphs[icentr][ieID][iconv].Fit("pol0","Q")
            #TMultigraphs[-1].Draw()
if not Save_to_Html: c0.Draw()

In [21]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [22]:
graphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(1,len(eIDbins)):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            raw_signal_frac =  hists_pt_real[0][ieID][iconv][icentr].Integral(5,10)/(hists_pt_real[0][0][iconv][icentr].Integral(5,10)+1e-6)
            raw_primary_frac = hists_pt_sim[0][ieID][iconv][icentr].Integral(5,10)/(hists_pt_sim[0][0][iconv][icentr].Integral(5,10)+1e-6)
            raw_hadron_frac =  hists_pt_sim[1][ieID][iconv][icentr].Integral(5,10)/(hists_pt_sim[1][0][iconv][icentr].Integral(5,10)+1e-6)
            #print(raw_signal_frac, raw_primary_frac, raw_hadron_frac)
            if (raw_signal_frac - raw_primary_frac) == 0: hadron_frac = 0
            else: 
                if raw_primary_frac - raw_hadron_frac > 0:
                    hadron_frac = ( raw_primary_frac - raw_signal_frac ) / ( raw_primary_frac - raw_hadron_frac )
                else: hadron_frac = -1
                if hadron_frac<0: hadron_frac = 0
                if hadron_frac>1: hadron_frac = 1
            graphs_eid.append(hadron_frac)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
if not Save_to_Html: print(graphs)

[[[0.4195904547373691, 0.5715471365587579, 0.579221160646508], [0.5784962716487986, 0.698124366217839, 0.7094294591159324], [0.5918974910989738, 0.7042710238303065, 0.7103834444109571], [0.5931969405468794, 0.6951846293556097, 0.7004684407300086]], [[0.24697367365181128, 0.38095738183901995, 0.3939697331349098], [0.42247107324332767, 0.5435614540078222, 0.5562414215474665], [0.4359895998398571, 0.5513065356008573, 0.5606901886421607], [0.4164881116107655, 0.5208451855591161, 0.5329521557362672]], [[0.0991211263718396, 0.20255268049281425, 0.21704860924976904], [0.26100662964922355, 0.36713786978542495, 0.382382139480314], [0.27856417436224346, 0.38083953346630445, 0.39721543882137733], [0.21825806513929535, 0.31206922879603727, 0.3266987572391998]], [[0, 0.03489972811038372, 0.048684885316141235], [0.12954190258949738, 0.21448427445975307, 0.231393909881539], [0.14217040619578683, 0.2208554428768639, 0.23538964648280902], [0.06813209100628774, 0.12917543085663846, 0.14271017151822826]]

In [23]:
#if not Save_to_Html: Tgraphs[0][3][1].GetFunction("pol2").Eval(1)

In [24]:
if True :
    only_pos = 1
    c0 = root.TCanvas(f"c0",f"c0",1400,500*(2-only_pos))
    N_centr_loc = N_centr - 5*only_pos
    c0.Divide(3,1*(2-only_pos))
    legends = []
    for icentr in range(3):
        for iconv in range(1):
            c0.cd(icentr+1+N_centr_loc*iconv)
            root.gPad.SetLogy()
                
            Format_Hist_total(hists_pt_real[0][refrerence][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=2,  Lwidth=4,  Lcolor=2,  offset_x=1, offset_y=1.1, title="",  Malpha=1,  Lalpha=1)
            
            hists_pt_real[0][refrerence][iconv][icentr].GetXaxis().SetRange(4,45)
            hists_pt_real[0][refrerence][iconv][icentr].Draw("H")
    
            for iptbin in range(1,1+hists_pt_real[0][0][iconv][icentr].GetNbinsX()):
                kek = 0
                if  Tgraphs[icentr][1][1].GetFunction("pol0") and icentr<5:
                    kek = Tgraphs[icentr][1][1].GetFunction("pol0").Eval(hists_pt_real[0][refrerence][iconv][icentr].GetBinCenter(iptbin))/1.5
                hists_pt_sim[1][0][iconv][icentr].SetBinContent(iptbin,kek*hists_pt_real[0][refrerence][iconv][icentr].GetBinContent(iptbin))
                #hists_pt_sim[1][1][iconv][icentr].SetBinContent(iptbin,Tgraphs[icentr][3][1].GetFunction("pol2").Eval(hists_pt_real[0][0][iconv][icentr].GetBinCenter(iptbin))*hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin))
                
            Format_Hist_total(hists_pt_sim[1][0][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=4,  Lcolor=1,  offset_x=1, offset_y=1.1, title="",  Malpha=1,  Lalpha=1)
            Format_Hist_total(hists_pt_sim[1][1][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1.1, title="",  Malpha=1,  Lalpha=1)
            hists_pt_sim[1][0][iconv][icentr].Draw('same H')
            legends.append(root.TLegend(0.4,0.68,0.9,0.98, f"Au+Au {central_bins[icentr*2]}-{central_bins[icentr*2+1]}%"))
            legends[-1].AddEntry(hists_pt_real[0][refrerence][iconv][icentr],"BDT basic","l")
            legends[-1].AddEntry(hists_pt_sim[1][0][iconv][icentr],"hadrons","l")
            Format_Legend(legends[-1])
            legends[-1].Draw()
            #hists_pt_sim[1][1][iconv][icentr].Draw('same H')
c0.Draw()         
c0.SaveAs("output/BDT_basic_hadron_fraction.png")   

In [25]:
if False:
    outfilename = f"output/files/sngl_spectra_{refrerence}.root"
    outfile = root.TFile.Open(outfilename, "recreate")
    for icentr in range(N_centr_loc):
        hists_pt_real[0][refrerence][0][icentr].Write()
        hists_pt_sim[1][0][0][icentr].Write()
    outfile.Close()

In [26]:
if True :
    best_type = 2
    only_pos = 1
    c0 = root.TCanvas(f"c0",f"c0",1400,1000*(2-only_pos))
    N_centr_loc = N_centr - 5*only_pos
    c0.Divide(3,2*(2-only_pos))
    hist_pt_orig_centr = []
    legends=[]
    for icentr in range(N_centr_loc):
        hist_pt_orig_centr.append(hist_pt_orig.ProjectionX(f"orig{icentr}",icentr+1,icentr+1,1,1))
        for iconv in range(1):
            c0.cd(icentr+1+N_centr_loc*iconv)
            root.gPad.SetLogy()
                
            Format_Hist_total(hists_pt_real[0][best_type][iconv][icentr], title_x="p_{T} [GeV]",  title_y="1/2#pip_{T} d^{2}N/dydp_{T}", left=0.3, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=1, Mcolor=2,  Lwidth=4,  Lcolor=2,  offset_x=1, offset_y=1.5, title="",  Malpha=1,  Lalpha=1)
            
            hists_pt_real[0][best_type][iconv][icentr].GetXaxis().SetRange(4,45)

            for iptbin in range(1,1+hists_pt_real[0][0][iconv][icentr].GetNbinsX()):
                bincontent = hists_pt_real[0][best_type][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][best_type][iconv][icentr].GetBinContent(iptbin)+1e-12)*hist_pt_orig_centr[icentr].GetBinContent(iptbin)
                error = hists_pt_real[0][best_type][iconv][icentr].GetBinContent(iptbin)**0.5/(hists_pt_sim[0][best_type][iconv][icentr].GetBinContent(iptbin)+1e-12)*hist_pt_orig_centr[icentr].GetBinContent(iptbin)
                bincontent /= 2*3.14*hists_pt_real[0][best_type][iconv][icentr].GetBinWidth(iptbin)*hists_pt_real[0][best_type][iconv][icentr].GetBinCenter(iptbin)
                error /= 3.14*hists_pt_real[0][best_type][iconv][icentr].GetBinWidth(iptbin)*hists_pt_real[0][best_type][iconv][icentr].GetBinCenter(iptbin)
                hists_pt_real[0][best_type][iconv][icentr].SetBinContent(iptbin,bincontent)
                hists_pt_real[0][best_type][iconv][icentr].SetBinError(iptbin,error)

                bincontent = hists_pt_real[0][best_type+1][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][best_type+1][iconv][icentr].GetBinContent(iptbin)+1e-12)*hist_pt_orig_centr[icentr].GetBinContent(iptbin)
                error = hists_pt_real[0][best_type+1][iconv][icentr].GetBinContent(iptbin)**0.5/(hists_pt_sim[0][best_type+1][iconv][icentr].GetBinContent(iptbin)+1e-12)*hist_pt_orig_centr[icentr].GetBinContent(iptbin)
                bincontent /= 2*3.14*hists_pt_real[0][best_type][iconv][icentr].GetBinWidth(iptbin)*hists_pt_real[0][best_type][iconv][icentr].GetBinCenter(iptbin)
                error /= 3.14*hists_pt_real[0][best_type][iconv][icentr].GetBinWidth(iptbin)*hists_pt_real[0][best_type][iconv][icentr].GetBinCenter(iptbin)
                hists_pt_real[0][best_type+1][iconv][icentr].SetBinContent(iptbin,bincontent)
                hists_pt_real[0][best_type+1][iconv][icentr].SetBinError(iptbin,error)
                
            
            hists_pt_real[0][best_type][iconv][icentr].Scale(1./2.413e9)
            hists_pt_real[0][best_type+1][iconv][icentr].Scale(1./2.413e9)
            Format_Hist_total(hists_pt_real[0][best_type+1][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.21, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=20,  Msize=1, Mcolor=4,  Lwidth=4,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            
            legends.append(root.TLegend(0.4,0.7,0.9,0.98, "e^{+} "+f"Au+Au {central_bins[icentr*2]}-{central_bins[icentr*2+1]}%"))
            legends[-1].AddEntry(hists_pt_real[0][best_type][iconv][icentr],"bdt loose","l")
            legends[-1].AddEntry(hists_pt_real[0][best_type+1][iconv][icentr],"bdt tight","l")
            Format_Legend(legends[-1])
            hists_pt_real[0][best_type][iconv][icentr].Draw()
            hists_pt_real[0][best_type+1][iconv][icentr].Draw("same")

            legends[-1].Draw()
    c0.Draw()  
    #c0.SaveAs("output/ptspectrum__loose_tight.png")          

In [27]:
if False:
    outfilename = f"output/files/reco_spectra.root"
    outfile = root.TFile.Open(outfilename, "recreate")
    for icentr in range(N_centr_loc):
        hists_pt_real[0][best_type][0][icentr].Write()
        hists_pt_real[0][best_type+1][0][icentr].Write()
    outfile.Close()